In [ ]:
#Emailing
#0. Import libraries for dataframe and email specific functions
import email, smtplib, ssl, csv, pandas, codecs, PyPDF2, os, openpyxl
import numpy as np
import pandas as pd
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email import encoders


#1. Set up HTML signature; paste html in or add link to HTML signature document via a variable stored as html
html = """
#paste signature here! 
"""

#2. Set up dataframes:
# 2.1 Making XMLX participant file usable
# 2.2 Parsing PDF certs for participant names


#2.1. Convert XMLX Emails file to Dataframe usable object
excel = openpyxl.load_workbook("Documents/certs.xlsx")
sheet = excel.active
col = csv.writer(open("tt.csv",'w', newline=""))
for r in sheet.rows:
    col.writerow([cell.value for cell in r]) 
#Dataframe usable object from XMLX stored as df
df = pandas.DataFrame(pandas.read_csv("tt.csv"))
#Reformat dataframe
df_new = df.rename(columns={'First name': 'First','Last name': 'Last'})
df_perfect = df_new[["First", "Last",'Email']]


#2.2 Parsing the PDF Certs for paritcipant names to match to their emails
extracted_names = []

def parsernames(pdf_file):
    # creating a pdf file object
    pdfFileObj = open(('Documents/certs/'+pdf_file), 'rb') 
    # creating a pdf reader object 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
    # printing number of pages in pdf file 
    # extracting text from page 
    pageObj = pdfReader.getPage(0)
    x = pageObj.extractText()
    #string_x = str(x)
    y = ((x)[155:185])
    z = ((y) + ' ' + pdf).split()
    #print(z)
    for item in z:
        if item.startswith('Atte') or item.startswith('th'):
            z.remove(item)
            #print(z)
    m = (z[0],z[1],z[-1])
    #print(m)
    #os.rename(('Documents/certs/'+pdf_file),('Documents/certs/'+(str(z[0])+'_'+str(z[1])+'.pdf'))
    extracted_names.append(m)
    pdfFileObj.close()
    
path = 'Documents/certs/'
files = os.listdir(path)
for pdf in files:
    parsernames(pdf)

#2.3 Reformat dataframe    
extracted_names
extracted_names_df = pd.DataFrame(extracted_names,columns=['First','Last','pdf'])
#extracted_names_df = extracted_names_df.rename(columns={'0': 'First', '1': 'Last', '2': 'pdf'})
extracted_names_df

#2.4 Merge dataframes
result = pd.merge(df_perfect,
                  extracted_names_df[['First', 'Last','pdf']],
                  on='First')

#2.5 Convert to CSV
result.to_csv('Documents/result.csv')


#Content of the result dataframe/CSV
#1= First 
#2 = Last_x 
#3 = Email 
#4 = Last_y 
#5 = PDF 

#3. Email function, to be looped at end
def my_function(First,filename,recievername):
    subject = "Certificate of Attendance for ELIXIR Data-Interoperability Joint Platform Meeting 23-25 November 2021"
    body = """Dear """+First+""",

    Thank you for your attendance at the ELIXIR Data-Interoperability Joint Platform Meeting which took place on the 23-25 of November 2021. Please find attached your certificate of attendance and apologies for the delay. 

    Sincerely,
    Gavin Farrell"""

    #Change to alias!
    sender_email = input("Type your email and press enter:") 
    password = input("Type your password and press enter:")
    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = recievername
    message["Subject"] = subject
    message["Bcc"] = recievername  # Recommended for mass emails
    # Add body to email
    message.attach(MIMEText(body, "plain"))
    # Open PDF file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)
    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename[16:]}",
    )

    part2 = MIMEText(html, "html")

    # Add attachment to message and convert message to string
    message.attach(part)
    message.attach(part2)
    message = message.as_string()

    #Send emails by looping throygh info in the dataframe
    # Log in to server using secure context and send email
    port = 587  # For starttls
    smtp_server = "outgoing.ebi.ac.uk"
    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, port) as server:
        server.ehlo()  # Can be omitted
        server.starttls(context=context)
        server.ehlo()  # Can be omitted
        server.login(sender_email, password)
        server.sendmail(
                    sender_email,
                    recievername,
                    message#.format(First=row[1])
                )
            
with open("Documents/test_mails.csv") as file:
            reader = csv.reader(file)
            next(reader)  # Skip header row
            for row in reader:
                my_function(row[1],('Documents/certs/'+row[5]),row[3])
                print('Email sent successfully to: '+row[1]+' '+row[2]+' at '+ row[3])
                
#Confirmation fo script execution                
print('All Emails successfuly sent')

#Issues to be fixed:
#1. Joining by first name issue if too many with first names; solution = 
#2. Work in 3rd names
#3. Duplicates? How does this affect
#4. Alias for ELIXIR Mailer; solution = change to yagmail instead of smtplib?... 
#5. Check mergers with 'result' object
#6. Add input buttons
#email/password/signature link to file